In [1]:
import joblib

from src.utils.paths import DATA_DIR, MODELS_DIR

import pandas as pd
import numpy as np


In [2]:
model = joblib.load(MODELS_DIR / "best_model_v2.pkl")
metadata = joblib.load(MODELS_DIR / "model_metadata_v2.pkl")


In [3]:
print('Information about model:')
print(f"Name = {metadata['model_name']}")
print(f'R2 (test) = {metadata['test_r2']}')


Information about model:
Name = GradientBoosting
R2 (test) = 0.81


In [ ]:
data = pd.read_csv(DATA_DIR / "data_v4.csv")
data.head(3)


,job_category,seniority_level,english_level,experience_years,salary_usd
0,Software Engineering,C-level,Advanced,18.0,8000.0
1,Software Engineering,C-level,Advanced,14.0,14000.0
2,Software Engineering,C-level,Advanced,10.0,14664.0


In [ ]:
X_test = data.drop(columns=['salary_usd'])


In [58]:
def test_model(index):
    sample = X_test.iloc[index]
    sample = pd.DataFrame([sample])

    predict_value = model.predict(sample)
    print(f'Input data: {sample.iloc[0].to_dict()}')
    print(f'Predicted value: {np.round(predict_value[0], 0)}')
    print(f'Real value: {data['salary_usd'][index]}')


In [ ]:
test_model(index=1100)


Input data: {'job_category': 'QA', 'seniority_level': 'Senior', 'english_level': 'Upper-Intermediate', 'experience_years': 9.0}
Predicted value: 3955.0
Real value: 4250.0


In [65]:
model


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_scaled',
                                                  StandardScaler(),
                                                  Index(['experience_years'], dtype='object')),
                                                 ('target_scaled',
                                                  Pipeline(steps=[('encoder',
                                                                   TargetEncoder()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['seniority_level']),
                                                 ('frequency_scaled',
                                                  Pipeline(steps=[('encoder',
                                                                   FrequencyEncoder()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['job_category']),
                                                 ('ordinal_scaled',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['english_level'])])),
                ('regressor',
                 GradientBoostingRegressor(learning_rate=0.05,
                                           min_samples_split=10,
                                           random_state=25))])

In [67]:
# creating configs for input data
config = {
    'category': data['job_category'].unique().tolist(),
    'title': data['seniority_level'].unique().tolist(),
    'english_level': data['english_level'].unique().tolist(),
}

config


{'category': ['Software Engineering',
  'QA',
  'Marketing',
  'Product & Project management',
  'Design',
  'Data Science & Machine Learning',
  'HR',
  'IT Operations',
  'Business',
  'Security'],
 'title': ['C-level',
  'Intern',
  'Junior',
  'Lead',
  'Management',
  'Middle',
  'Senior',
  'Staff+'],
 'english_level': ['Advanced',
  'Intermediate',
  'Upper-Intermediate',
  'Pre-Intermediate',
  'Elementary']}

In [69]:
import os
import json
with open('configs/allowed_values.json', 'w') as f:
    json.dump(config, f, indent=2)
print('Config created')


Config created
